In [127]:
import pandas as pd

In [128]:
def convert_to_float(value):
    # Remove o símbolo R$, qualquer ponto de milhar, substitui a vírgula decimal por ponto, e remove espaços
    value = value.replace('R$', '').replace('.', '').replace(',', '.').replace(' ', '')
    return float(value)

# Função para converter valores percentuais
def convert_to_percent(value):
    # Remove o símbolo %, substitui a vírgula decimal por ponto, e remove espaços
    return float(value.replace('%', '').replace(',', '.').replace(' ', ''))

TIPO DE INVESTIMENTO | EMISSOR | VALOR APLICADO | DATA VENCIMENTO |	DATA APLICAÇÃO | INDEXADOR | TAXA |	VALOR BRUTO | QUANTIDADE


In [129]:
seqcol = ['TIPO DE INVESTIMENTO', 'EMISSOR', 'VALOR APLICADO', 'DATA VENCIMENTO', 'DATA APLICAÇÃO', 'INDEXADOR', 'TAXA (%)', 'VALOR BRUTO', 'QUANTIDADE', 'CORRETORA']

## NUINVEST

In [130]:
nuinvest = pd.read_csv("E:/dados_inv/nuinvest.csv", encoding='iso-8859-1', delimiter=';', header=1, decimal=',',
                       converters={
                            'VALOR APLICADO': convert_to_float,
                            'VALOR BRUTO': convert_to_float,
                            'VALOR LÍQUIDO': convert_to_float,
                            'RENDIMENTO R$': convert_to_float,
                            'QUANTIDADE': lambda x: float(x.replace('.', '').replace(',', '.')),
                            'RENDIMENTO %': convert_to_percent
                        },
                        parse_dates=['VENCIMENTO'], 
                        dayfirst=True  # Especifica que o formato de data é dia/mês/ano)
                    )

#Extrai indexador e taxa
nuinvest['INDEXADOR'] = nuinvest['TAXA NEGOCIADA'].str.extract('(CDI|IPC-A)')
nuinvest['TAXA (%)'] =  nuinvest['TAXA NEGOCIADA'].str.extract('(\d+,\d+|\d+)%')

#Remove colunas indesejadas
nuinvest = nuinvest.drop(columns=['TAXA NEGOCIADA', 'VALOR LÍQUIDO', 'RENDIMENTO R$', 'RENDIMENTO %'])

#renomeia as colunas para o padrão desejado
nuinvest.rename(columns={'DESCRIÇÃO': 'EMISSOR',
                         'VENCIMENTO': 'DATA VENCIMENTO'}, inplace=True)

#cria o campo de data da aplicação
nuinvest['DATA APLICAÇÃO'] = pd.to_datetime(pd.Series([None] * len(nuinvest)))

#cria a coluna da corretora
nuinvest['CORRETORA'] = 'NUINVEST'

#reordena as colunas conforme padrão desejado
nuinvest = nuinvest.reindex(columns=seqcol)

display(nuinvest)

,TIPO DE INVESTIMENTO,EMISSOR,VALOR APLICADO,DATA VENCIMENTO,DATA APLICAÇÃO,INDEXADOR,TAXA (%),VALOR BRUTO,QUANTIDADE,CORRETORA
0,LC,ZEMA FINANCEIRA,3934.68,2024-06-14,NaT,CDI,125,4811.76,3.0,NUINVEST
1,CDB,BANCO BMG,2000.00,2024-06-17,NaT,IPC-A,"6,33",2448.41,2000.0,NUINVEST
2,LC,ZEMA FINANCEIRA,1374.81,2024-06-26,NaT,CDI,125,1600.06,1.0,NUINVEST
3,LC,CARUANA,4361.68,2024-07-05,NaT,CDI,125,5226.70,3.0,NUINVEST
4,LC,CARUANA,7269.47,2024-07-05,NaT,CDI,125,8711.18,5.0,NUINVEST
...,...,...,...,...,...,...,...,...,...,...
74,CDB,PAGBANK,1105.94,2027-01-20,NaT,CDI,125,1125.91,1130.0,NUINVEST
75,CDB,PAGBANK,4890.99,2027-01-21,NaT,CDI,125,4979.28,5000.0,NUINVEST
76,CDB,PAGBANK,970.99,2027-02-05,NaT,CDI,125,991.99,1000.0,NUINVEST
77,CDB,BANCO DIGIMAIS,1667.47,2027-03-19,NaT,CDI,125,1697.57,1707.0,NUINVEST


In [131]:
nuinvest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   TIPO DE INVESTIMENTO  79 non-null     object        
 1   EMISSOR               79 non-null     object        
 2   VALOR APLICADO        79 non-null     float64       
 3   DATA VENCIMENTO       79 non-null     datetime64[ns]
 4   DATA APLICAÇÃO        0 non-null      datetime64[ns]
 5   INDEXADOR             76 non-null     object        
 6   TAXA (%)              79 non-null     object        
 7   VALOR BRUTO           79 non-null     float64       
 8   QUANTIDADE            79 non-null     float64       
 9   CORRETORA             79 non-null     object        
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 6.3+ KB


## XP

In [132]:
xp = pd.read_excel('E:/dados_inv/xp.xlsx', skiprows=7)

xp.rename(columns={xp.columns[0]: 'TIPO DE INVESTIMENTO'}, inplace=True)

#remove linhas na coluna 'TIPO DE INVESTIMENTO' que começam com número.
#necessário com base no formato de dados fornecido pela XP
xp = xp[~xp['TIPO DE INVESTIMENTO'].str.match(r'^\d')]
xp = xp[xp['TIPO DE INVESTIMENTO'] != ' ']

# divide a coluna tipo de investimento entre eo tipo e o emissor do título
padrao = r'^([A-Z]{3}).*?([^-]*)-'
xp[['TIPO DE INVESTIMENTO', 'EMISSOR']] = xp['TIPO DE INVESTIMENTO'].str.extract(padrao)

# #extrai o indexador e a taxa
xp['INDEXADOR'] = xp['Taxa a mercado'].str.extract('(CDI|IPC-A)')
xp['TAXA (%)'] = xp['Taxa a mercado'].str.extract('(\d+,\d+|\d+)%')


# # converte colunas monetárias para float
xp['VALOR APLICADO'] = xp["Valor aplicado"].map(convert_to_float)
xp['VALOR BRUTO'] = xp['Posição a mercado'].map(convert_to_float)

#formata coluna e datas para o formato esperado
xp['DATA VENCIMENTO'] = pd.to_datetime(xp['Data vencimento'], format='%d/%m/%Y')
xp['DATA APLICAÇÃO'] = pd.to_datetime(xp['Data aplicação'], format='%d/%m/%Y')

#formata qtd e converte para float
xp['QUANTIDADE'] = pd.to_numeric(xp['Quantidade'])

#cria a coluna da corretora
xp['CORRETORA'] = 'XP'

# #remove colunas desnecessárias
xp = xp.drop(columns=['Posição a mercado', 'Taxa a mercado', 'Quantidade',
                      '% Alocação', 'Data vencimento', 'Data aplicação',
                      'Valor aplicado', 'Preço Unitário'])#,
                    #   'Preço Unitário.1', 'Quantidade.1', 
                    #   'Preço Unitário.2', 'Quantidade.2',
                    #   'Preço Unitário.3', 'Quantidade.3'])

#reordena colunas
xp = xp.reindex(columns=seqcol)

#converter  valores para float
display(xp.head(10))

,TIPO DE INVESTIMENTO,EMISSOR,VALOR APLICADO,DATA VENCIMENTO,DATA APLICAÇÃO,INDEXADOR,TAXA (%),VALOR BRUTO,QUANTIDADE,CORRETORA
0,CDB,NEON FINANCEIRA,6895.46,2026-04-06,2024-05-17,CDI,"121,00",6928.29,6,XP
1,CDB,PERNAMBUCANAS FINANCIADOR,5514.62,2025-09-28,2023-12-27,CDI,"125,00",5828.66,4,XP
2,CDB,NEON FINANCEIRA,4596.97,2026-04-06,2024-05-17,CDI,"121,00",4618.86,4,XP
3,CDB,NEON FINANCEIRA,2292.39,2025-10-06,2024-05-17,CDI,"120,86",2303.29,2,XP
4,CDB,PERNAMBUCANAS FINANCIADOR,1279.00,2025-08-15,2024-05-17,CDI,"120,61",1285.07,1,XP
5,CDB,PERNAMBUCANAS FINANCIADOR,1175.37,2025-11-17,2023-12-26,CDI,"128,80",1245.10,1,XP
8,CDB,BANCO VOITER S.A.,8000.00,2026-04-10,2024-04-10,IPC-A,"6,85",8112.04,8,XP
9,CDB,BANCO PAN S/A,4000.00,2025-03-16,2021-03-17,IPC-A,"4,75",5701.05,4,XP
10,CDB,BANCO C6 CONSIGNADO S.A.,3000.00,2024-09-16,2021-09-17,IPC-A,"5,45",4066.23,3,XP
11,CRA,JBS,3068.09,2024-10-15,2020-04-20,IPC-A,"6,74",3953.48,3,XP


In [133]:
xp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 11
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   TIPO DE INVESTIMENTO  10 non-null     object        
 1   EMISSOR               10 non-null     object        
 2   VALOR APLICADO        10 non-null     float64       
 3   DATA VENCIMENTO       10 non-null     datetime64[ns]
 4   DATA APLICAÇÃO        10 non-null     datetime64[ns]
 5   INDEXADOR             10 non-null     object        
 6   TAXA (%)              10 non-null     object        
 7   VALOR BRUTO           10 non-null     float64       
 8   QUANTIDADE            10 non-null     int64         
 9   CORRETORA             10 non-null     object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 880.0+ bytes


## ATIVA

In [134]:
ativa = pd.read_excel('E:/dados_inv/ativa.xls', sheet_name='Renda Fixa Privada')
ativa = ativa.dropna()

#calcula valor aplicado
ativa['VALOR APLICADO'] = ativa['PU de Compra'] * ativa ['Quantidade']

#cria colunas de data aplicação e data de vencimento
ativa['DATA APLICAÇÃO'] = pd.to_datetime(ativa['Data'])
ativa['DATA VENCIMENTO'] = pd.to_datetime(ativa['Unnamed: 5'])

# #extrai o indexador e a taxa
ativa['INDEXADOR'] = ativa['Remuneração'].str.extract('(CDI|IPC-A)')
extracoes = ativa['Remuneração'].str.extract(r'(\d+)% CDI(?: \+ (\d+\.?\d*) a.a)?|IPC-A \+ (\d+\.?\d*) a.a')
ativa['TAXA (%)'] = extracoes.apply(lambda row: row[2] if pd.isna(row[0]) and pd.isna(row[1]) else (row[1] if pd.notna(row[1]) else row[0]), axis=1)

#exclui colunas desnecessárias
ativa = ativa.drop(columns=['Ticker', 'Data', 'Unnamed: 5', 'PU de Compra',
                            'Remuneração', 'PU de Referência', 'PU atual'])

# #renomeia as colnas conforme padrão
ativa.rename(columns={'Títulos': 'TIPO DE INVESTIMENTO',
                   'Emissor': 'EMISSOR',
                   'Emissor': 'EMISSOR',
                   'Quantidade': 'QUANTIDADE',
                   'Total': 'VALOR BRUTO'}, inplace=True)

#reordena as colunas conforme padrão

#cria a coluna da corretora
ativa['CORRETORA'] = 'ATIVA'

ativa = ativa.reindex(columns=seqcol)

display(ativa)

,TIPO DE INVESTIMENTO,EMISSOR,VALOR APLICADO,DATA VENCIMENTO,DATA APLICAÇÃO,INDEXADOR,TAXA (%),VALOR BRUTO,QUANTIDADE,CORRETORA
1,CDB,BANCO BMG S/A,3000.0,2025-09-01,2023-08-31,CDI,1.85,3296.28,3000.0,ATIVA
2,CDB,BANCO VOITER,8000.0,2026-04-13,2024-04-22,IPC-A,7.6,8090.64,8.0,ATIVA
3,LCA,BANCO VOITER,8000.0,2025-01-30,2024-04-29,CDI,96,8067.10,8.0,ATIVA
4,CDB,BANCO VOITER,5000.0,2027-04-12,2024-04-10,IPC-A,7.25,5072.61,5.0,ATIVA
5,CDB,BANCO VOITER,2000.0,2026-04-01,2024-04-10,IPC-A,7,2037.63,2.0,ATIVA
6,CDB,BANCO VOITER,5000.0,2026-04-01,2024-04-10,IPC-A,7,5094.09,5.0,ATIVA
7,CDB,BANCO VOITER,5000.0,2026-10-16,2024-04-29,CDI,123,5053.79,5.0,ATIVA


In [135]:
ativa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 1 to 7
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   TIPO DE INVESTIMENTO  7 non-null      object        
 1   EMISSOR               7 non-null      object        
 2   VALOR APLICADO        7 non-null      float64       
 3   DATA VENCIMENTO       7 non-null      datetime64[ns]
 4   DATA APLICAÇÃO        7 non-null      datetime64[ns]
 5   INDEXADOR             7 non-null      object        
 6   TAXA (%)              7 non-null      object        
 7   VALOR BRUTO           7 non-null      float64       
 8   QUANTIDADE            7 non-null      float64       
 9   CORRETORA             7 non-null      object        
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 616.0+ bytes


## BTG

In [136]:
btg = pd.read_excel('E:/dados_inv/btg.xlsx', decimal=',')
nomes_colunas = ['Produto',	'Vencimento', 'Taxa de aplicação',	'Quantidade', 'Preço (R$)', 'Saldo Líquido (R$)', 'Saldo Bruto (R$)',	'Operações']

#exclui coluna operações
btg = btg.drop(columns=['Operações', 'Saldo Líquido (R$)', 'Preço (R$)'])

#o que não converter como número é lixo e será excluido
btg['QUANTIDADE'] = pd.to_numeric(btg['Quantidade'], errors='coerce')

#exclui colunas com NaN
btg = btg.dropna()

#converte valores monetários para float
btg['VALOR BRUTO'] = pd.to_numeric(btg['Saldo Bruto (R$)'])

#converte valores de data
btg['DATA VENCIMENTO'] = pd.to_datetime(btg['Vencimento'])
btg['DATA APLICAÇÃO'] = pd.to_datetime(pd.Series([None] * len(btg)))

# divide a coluna tipo de investimento entr eo tipo e o emissor do título
padrao = r'^([A-Z]{3}).*?((?:(?!S\.).)*)'
btg[['TIPO DE INVESTIMENTO', 'EMISSOR']] = btg['Produto'].str.extract(padrao)

# #extrai o indexador e a taxa
btg['Taxa de aplicação'] = btg['Taxa de aplicação'].str.replace(',' , '.')

btg['INDEXADOR'] = btg['Taxa de aplicação'].str.extract('(CDI|IPCA)')
extracoes = btg['Taxa de aplicação'].str.extract(r'(\d+).\d+% do CDI|CDI \+ (\d+\.?\d*)%|IPCA \+ (\d+\.?\d*)%')
btg['TAXA (%)'] = extracoes.apply(lambda row: row[2] if pd.isna(row[0]) and pd.isna(row[1]) else (row[1] if pd.notna(row[1]) else row[0]), axis=1)

#para que o indice corresponda ao outros dfs
btg['INDEXADOR'] = btg['INDEXADOR'].replace('IPCA', 'IPC-A')

#apaga colunas não mais ecessárias
btg = btg.drop(columns=['Produto', 'Taxa de aplicação', 'Vencimento', 'Saldo Bruto (R$)'])

#cria a coluna da corretora
btg['CORRETORA'] = 'BTG'

#reorganiza colunas
btg = btg.reindex(columns=seqcol)

display(btg)

,TIPO DE INVESTIMENTO,EMISSOR,VALOR APLICADO,DATA VENCIMENTO,DATA APLICAÇÃO,INDEXADOR,TAXA (%),VALOR BRUTO,QUANTIDADE,CORRETORA
0,CDB,LEBES FINANCEIRA,NaN,2025-08-04,NaT,CDI,123,7844.68,6.0,BTG
2,CDB,BANCO BMG,NaN,2027-04-26,NaT,IPC-A,7.35,2022.53,2.0,BTG
6,CRA,MADERO,NaN,2027-03-15,NaT,CDI,2.60,796.27,1.0,BTG


In [137]:
btg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 6
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   TIPO DE INVESTIMENTO  3 non-null      object        
 1   EMISSOR               3 non-null      object        
 2   VALOR APLICADO        0 non-null      float64       
 3   DATA VENCIMENTO       3 non-null      datetime64[ns]
 4   DATA APLICAÇÃO        0 non-null      datetime64[ns]
 5   INDEXADOR             3 non-null      object        
 6   TAXA (%)              3 non-null      object        
 7   VALOR BRUTO           3 non-null      float64       
 8   QUANTIDADE            3 non-null      float64       
 9   CORRETORA             3 non-null      object        
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 264.0+ bytes


## Concatena os DFs de cada corretora

In [141]:
todos = pd.concat([nuinvest, xp, ativa, btg], axis=0)
todos = todos.reset_index(drop=True)
display(todos)

todos.to_excel('E:/dados_inv/resumo.xlsx', index=False)

,TIPO DE INVESTIMENTO,EMISSOR,VALOR APLICADO,DATA VENCIMENTO,DATA APLICAÇÃO,INDEXADOR,TAXA (%),VALOR BRUTO,QUANTIDADE,CORRETORA
0,LC,ZEMA FINANCEIRA,3934.68,2024-06-14,NaT,CDI,125,4811.76,3.0,NUINVEST
1,CDB,BANCO BMG,2000.00,2024-06-17,NaT,IPC-A,"6,33",2448.41,2000.0,NUINVEST
2,LC,ZEMA FINANCEIRA,1374.81,2024-06-26,NaT,CDI,125,1600.06,1.0,NUINVEST
3,LC,CARUANA,4361.68,2024-07-05,NaT,CDI,125,5226.70,3.0,NUINVEST
4,LC,CARUANA,7269.47,2024-07-05,NaT,CDI,125,8711.18,5.0,NUINVEST
...,...,...,...,...,...,...,...,...,...,...
94,CDB,BANCO VOITER,5000.00,2026-04-01,2024-04-10,IPC-A,7,5094.09,5.0,ATIVA
95,CDB,BANCO VOITER,5000.00,2026-10-16,2024-04-29,CDI,123,5053.79,5.0,ATIVA
96,CDB,LEBES FINANCEIRA,NaN,2025-08-04,NaT,CDI,123,7844.68,6.0,BTG
97,CDB,BANCO BMG,NaN,2027-04-26,NaT,IPC-A,7.35,2022.53,2.0,BTG


In [139]:
todos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   TIPO DE INVESTIMENTO  99 non-null     object        
 1   EMISSOR               99 non-null     object        
 2   VALOR APLICADO        96 non-null     float64       
 3   DATA VENCIMENTO       99 non-null     datetime64[ns]
 4   DATA APLICAÇÃO        17 non-null     datetime64[ns]
 5   INDEXADOR             96 non-null     object        
 6   TAXA (%)              99 non-null     object        
 7   VALOR BRUTO           99 non-null     float64       
 8   QUANTIDADE            99 non-null     float64       
 9   CORRETORA             99 non-null     object        
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 7.9+ KB
